### Topic Modeling for the SemEval corpus

In [231]:
from gensim import corpora
from gensim import models
from gensim.parsing.preprocessing import stem_text
from gensim.utils import SaveLoad, simple_preprocess

from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer 
import string

In [190]:
dictionary = corpora.Dictionary.load(folder + 'dictionary_new_stem.dict')
print(dictionary.num_docs, dictionary.num_pos, dictionary.num_nnz)

750000 481364752 206467698


In [191]:
stopwords = nltk.corpus.stopwords.words('english')
pprint.pprint(stopwords)

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [192]:
import string
def filterBadToken(token):
    for c in token:
        if c.isdigit() or c in string.punctuation:
            return True
    if token in stopwords or token in wordlist:
        return True
    return False

In [193]:
dct = dictionary.token2id
badIds = []
for k,v in dct.items():
    if filterBadToken(k):
        badIds.append(v)
        print(k, dictionary.dfs[v])
print('total tokens:', len(dct))

after 370691
doesn 116853
don 203290
her 170367
how 264061
if 362488
most 275209
my 161682
other 370212
she 182477
then 219006
where 228501
your 144399
both 184429
just 318091
our 268384
those 258748
while 271163
am 61638
off 190587
these 240279
until 112245
against 214328
below 140017
clinton 57673
didn 124231
do 310139
each 116131
here 209100
no 342478
so 347421
than 369463
there 365957
trump 107931
under 180663
what 363781
whom 39684
won 112806
you 316116
can 362531
couldn 35876
few 144186
had 350783
him 187435
over 337396
such 199299
them 291967
through 202314
people 116747
some 330237
between 177928
isn 87738
itself 67950
now 299378
re 36908
same 184013
before 31617
being 8709
did 193516
down 213477
me 136810
they 117157
too 148782
one 116610
into 312739
why 151695
again 116253
doing 14728
himself 67181
should 209841
this 79583
once 12083
further 80887
haven 35541
own 201588
wasn 71115
having 1032
ll 90142
very 8469
was 31225
wouldn 42957
does 14562
shouldn 24224
hasn 34401
hersel

In [194]:
dictionary.filter_tokens(bad_ids = badIds)

In [201]:
dictionary.id2token = {v: k for (k, v) in dictionary.token2id.items()}

In [170]:
wordlist = ['trump', 'clinton', 'one', 'people']

In [202]:
import pprint
most_frequent_ids = (v for v in dictionary.token2id.values())
most_frequent_ids = sorted(most_frequent_ids, key=dictionary.dfs.get, reverse = True)
pprint.pprint([(dictionary.id2token[idx], dictionary.dfs[idx]) for idx in most_frequent_ids[:10]])

[('relat', 74908),
 ('event', 74788),
 ('fire', 74516),
 ('data', 74467),
 ('whose', 74430),
 ('short', 74412),
 ('legal', 74358),
 ('especi', 74287),
 ('anti', 74193),
 ('north', 73970)]


In [203]:
dictionary.filter_extremes(no_below=100, no_above=0.1)

In [204]:
len(dictionary.token2id)

43525

In [70]:
from gensim.utils import SaveLoad, simple_preprocess
from gensim.matutils import corpus2csc

class Bow(object):
    '''
    compute Bag-of-Word vectors using iterator
    '''
    def __init__(self, fileList, dct, stem):
        self.file = fileList
        self.dictionary = dct
        self.stem = stem
        
    def __iter__(self):
        if isinstance(self.file, str):
            self.file = [self.file]
        for f in self.file:
            for line in open(f, encoding='utf-8'):
                text = line.split('::')[1] + ' ' + line.split('::')[2]
                yield self.dictionary.doc2bow(simple_preprocess(text) )
                

In [205]:
folder = '../Feature_eng/tmp/'
name = 'articles-training-bypublisher'
corpus = Bow('../data/articles-training-byarticle.txt', dictionary, True)

In [206]:
corpora.MmCorpus.serialize('corpus.mm', corpus)

In [207]:
corpus = corpora.MmCorpus('corpus.mm')
model = models.LdaModel(corpus, id2word=dictionary, num_topics=30)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\gensim\models\ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us

In [208]:
topics = model.print_topics(num_words=6)
for t in topics:
    print(t)

(28, '0.004*"house" + 0.003*"twitter" + 0.002*"according" + 0.002*"investigation" + 0.002*"democratic" + 0.002*"americans"')
(13, '0.006*"august" + 0.004*"woman" + 0.003*"class" + 0.003*"twitter" + 0.003*"police" + 0.003*"men"')
(14, '0.011*"police" + 0.006*"hillary" + 0.004*"news" + 0.003*"way" + 0.003*"house" + 0.002*"officers"')
(21, '0.005*"hillary" + 0.004*"twitter" + 0.003*"weinstein" + 0.003*"daily" + 0.003*"news" + 0.003*"police"')
(10, '0.005*"hillary" + 0.003*"says" + 0.003*"fbi" + 0.002*"california" + 0.002*"many" + 0.002*"investigation"')
(15, '0.003*"august" + 0.003*"political" + 0.003*"national" + 0.003*"men" + 0.002*"house" + 0.002*"many"')
(9, '0.005*"police" + 0.003*"article" + 0.003*"news" + 0.003*"hillary" + 0.003*"may" + 0.002*"eu"')
(22, '0.004*"fbi" + 0.004*"news" + 0.003*"called" + 0.003*"weinstein" + 0.003*"twitter" + 0.003*"political"')
(12, '0.007*"fbi" + 0.007*"hillary" + 0.003*"investigation" + 0.003*"going" + 0.003*"story" + 0.002*"comey"')
(3, '0.006*"mr" 

In [188]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
news = pyLDAvis.gensim.prepare(model,corpus, dictionary)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [189]:
news

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
21     0.012695  0.006479       1        1  8.862764
22     0.027529  0.006557       2        1  8.582071
23     0.010457  0.033143       3        1  6.882948
11     0.001440  0.009224       4        1  6.189849
7      0.011929  0.000504       5        1  5.554315
1      0.018604  0.008973       6        1  5.536853
14     0.009114 -0.001535       7        1  4.909844
2      0.016532  0.003277       8        1  4.654990
10     0.008758  0.002586       9        1  4.405962
5      0.006986  0.006220      10        1  4.336926
27     0.019825 -0.016442      11        1  3.763718
4      0.024200 -0.038748      12        1  3.640006
13    -0.001747  0.018682      13        1  3.017082
25     0.005849 -0.000619      14        1  2.720373
9      0.005559 -0.010761      15        1  2.507963
19    -0.002950 -0.011413      16        1  2.370739
16    -0.005732  0.019568      17        1  2.346570
17    -0.001403 -0.010653      18        1  2.199653
28    -0.010588  0.010370      19        1  1.922501
3     -0.003853 -0.014479      20        1  1.911462
6     -0.015487  0.020791      21        1  1.832283
18    -0.002639 -0.004989      22        1  1.778556
12     0.002990 -0.015287      23        1  1.680606
26    -0.013384  0.011444      24        1  1.439110
8     -0.008462  0.012408      25        1  1.378159
15    -0.011347  0.004474      26        1  1.345101
20    -0.032075 -0.001504      27        1  1.250045
24    -0.024992 -0.013579      28        1  1.168195
29    -0.012950 -0.016337      29        1  1.155069
0     -0.034857 -0.018353      30        1  0.656281, topic_info=      Category        Freq        Term       Total  loglift  logprob
term                                                                
4397   Default  101.000000        send  101.000000  30.0000  30.0000
584    Default  929.000000   president  929.000000  29.0000  29.0000
529    Default  189.000000        free  189.000000  28.0000  28.0000
2946   Default  449.000000      police  449.000000  27.0000  27.0000
64     Default  418.000000       obama  418.000000  26.0000  26.0000
5756   Default  522.000000     hillary  522.000000  25.0000  25.0000
8233   Default  285.000000         fbi  285.000000  24.0000  24.0000
2022   Default  122.000000  california  122.000000  23.0000  23.0000
659    Default  441.000000       white  441.000000  22.0000  22.0000
1802   Default  319.000000         man  319.000000  21.0000  21.0000
8789   Default   33.000000         sms   33.000000  20.0000  20.0000
2286   Default  260.000000       women  260.000000  19.0000  19.0000
553    Default  331.000000        left  331.000000  18.0000  18.0000
511    Default  418.000000      donald  418.000000  17.0000  17.0000
563    Default   96.000000     message   96.000000  16.0000  16.0000
1703   Default  425.000000        news  425.000000  15.0000  15.0000
985    Default   56.000000        text   56.000000  14.0000  14.0000
1301   Default  159.000000          mr  159.000000  13.0000  13.0000
907    Default  336.000000        many  336.000000  12.0000  12.0000
5819   Default  108.000000         nfl  108.000000  11.0000  11.0000
469    Default  343.000000     america  343.000000  10.0000  10.0000
49     Default  366.000000        know  366.000000   9.0000   9.0000
925    Default  187.000000       north  187.000000   8.0000   8.0000
4342   Default   68.000000     angeles   68.000000   7.0000   7.0000
7843   Default  107.000000      anthem  107.000000   6.0000   6.0000
15799  Default   67.000000         los   67.000000   5.0000   5.0000
11450  Default  146.000000       comey  146.000000   4.0000   4.0000
470    Default  365.000000    american  365.000000   3.0000   3.0000
3953   Default  281.000000       going  281.000000   2.0000   2.0000
616    Default  221.000000      states  221.000000   1.0000   1.0000
...        ...         ...         ...         ...   

In [ ]:
#infer new document
doc_lda = model[doc_bow]

In [211]:
import spacy

# Initialize spacy 'en' model, keeping only tagger component needed for lemmatization
nlp = spacy.load('en', disable=['parser', 'ner'])

sentence = "This is a smart SAMPLE!. Can you parse? or is it too RE"

# Parse the sentence using the loaded 'en' model object `nlp`
doc = nlp(sentence)

# Extract the lemma for each token and join
" ".join([token.lemma_ for token in doc])

'this be a smart sample!. can -PRON- parse ? or be -PRON- too re'

foot
['This', 'be', 'a', 'smart', 'SAMPLE', '!', '.', 'Can', 'you', 'parse', '?', 'or', 'be', 'it', 'too', 'RE']
